# Schema Translation with AgentCore Gateway Interceptor 

### Overview

This notebook shows how to use AgentCore Gateway for **Schema Translation** between existing service contracts and MCP tool schemas, so agents can call your APIs without any custom protocol glue. The goal is to give you a reusable pattern to normalize heterogeneous backend schemas into a clean, agent-friendly interface, while keeping your source systems unchanged.

### Why This Matters

As your agent expands, you may need to:

- Restrict which **tools** certain users can call  
- Control access to **sensitive actions** (cancelOrder, updateOrder, deleteOrder, etc.)  
- Filter or redact **semantic search results** based on user permissions  
- Show users **only the tools they are allowed to see**  
- Enforce custom authorization logic that goes beyond what JWT tokens provide  
- Apply **centralized governance** without modifying individual tools or runtimes  

Gateway interceptors provide a scalable, plug-and-play way to implement these controls **without modifying the agent, the runtime, or the MCP server**.  
You enforce policy **at the Gateway level**, where every request naturally flows through.

### Why Schema Translation Matters

When building AI agents that interact with multiple backend services, schema inconsistencies can create integration complexity and slow down agent development. Schema translation via the AgentCore Gateway allows you to normalize and unify diverse API schemas into a consistent MCP format that your agents can easily consume. 

This approach:

- Eliminates the need to modify existing APIs or backends while exposing them as MCP-compatible tools
- Centralizes protocol translation and schema evolution, reducing maintenance overhead
- Enables seamless integration of REST APIs, Lambda functions, and MCP servers behind a single gateway
- Provides a scalable, maintainable way to keep your agent’s view of tools clean and consistent

Gateway interceptors provide a scalable, plug-and-play way to implement schema translation **without modifying the agent, the runtime, or the MCP server**.  
You enforce policy **at the Gateway level**, where every request naturally flows through.

---

### What This Tutorial Covers

You will implement Schema Translation by:

1. 🛠️ **Create AgentCore Gateway with Request Gateway Interceptor** 
The Request interceptor has logic to transform the schemas in the request and to add any additional parameters. 

2. 🔍 **Register tools with the AgentCore Gateway**  
Register the inventory tool with the AgentCore Gateway 

3. 📋 **Invoke tool**  
When the tool is invoked,the request is intercepted and the schema translation is performed. 

---

### Why Use Gateway Interceptors?

Gateway interceptors allow you to add logic for:

- **Performing Schema Translation**
Seamlessly convert request and response schemas between heterogeneous backend APIs and the MCP format.

- **Normalizing Tool Interfaces**
Create a consistent, agent-friendly schema abstraction across diverse services without changing underlying APIs.

- **Adapt Protocols and Data Formats**
Modify requests or responses to bridge differences in expected parameters, field names, or data structures.

- **Centralize Translation Logic**
Manage schema evolution and transformations in one place to simplify maintenance and improve scalability.

Because interceptors operate at the Gateway layer, they enable consistent schema handling for any connected MCP server or runtime without altering individual tool implementations.

![schema-translation](images/schema-translation-request-interceptor.png)

---

### Tutorial Details

| Information              | Details                                                                                         |
|--------------------------|-------------------------------------------------------------------------------------------------|
| **Tutorial type**        | Interactive                                                                                     |
| **AgentCore components** | AgentCore Gateway, Gateway Interceptors (Request)                                               |
| **Gateway Target type**  | Lambda Function                                                                                 |
| **Interceptor types**    | AWS Lambda (request)                                                                            |
| **Inbound Auth IdP**     | Amazon Cognito (CUSTOM\_JWT authorizer)                                                         |
| **Access Control**       | IAM to invoke lambda tool                                                                       |
| **Tutorial components**  | Gateway, Cognito, Gateway Interceptors, MCP tools (lambda)                                      |
| **Tutorial vertical**    | Cross-vertical                                                                                  |
| **Example complexity**   | Easy–Intermediate                                                                               |
| **SDK used**             | boto3                                                                                           |

---

### Prerequisites

To execute this tutorial you will need:

- Jupyter notebook (Python kernel)
- AWS credentials with permissions for:
  - Lambda
  - IAM
  - Cognito
  - AgentCore services (control plane + runtime)
- Python 3.13 or higher
- Basic understanding of AWS Lambda, IAM roles, Cognito, and AgentCore Gateway

> ⚠️ **Note:** The Cleanup section at the end deletes the AWS resources created by this tutorial (Gateway, Lambdas, IAM roles, etc.). Only run it when you’re ready to tear everything down.

---

## Part 1: Setup & Deployment

### Step 1.1: Install Dependencies

In [ ]:
!pip install -r requirements.txt

### Step 1.2: Import Required Libraries

In [ ]:
import boto3
import json
import time
import zipfile
import io
import sys
from pathlib import Path
from datetime import datetime
from botocore.exceptions import ClientError

# Add parent directory to path for utils
utils_dir = Path.cwd().parent
sys.path.insert(0, str(utils_dir))

import utils

print("✓ Libraries imported")

# Generate unique identifier for this deployment
DEPLOYMENT_ID = datetime.now().strftime('%Y%m%d-%H%M%S')
print(f"\nDeployment ID: {DEPLOYMENT_ID}")

### Step 1.3: Configure Deployment Variables

In [ ]:
# Configuration

LAMBDA_REGION = "us-east-1"  
REQUEST_LAMBDA_FUNCTION_NAME = f"request_interceptor-lambda-{DEPLOYMENT_ID}"
LAMBDA_ROLE_NAME = f"interceptor-lambda-role-{DEPLOYMENT_ID}"
GATEWAY_NAME = f"interceptor-gateway-{DEPLOYMENT_ID}"

print("Configuration:")
print(f" Request Lambda Function: {REQUEST_LAMBDA_FUNCTION_NAME}")
print(f"  Lambda Role: {LAMBDA_ROLE_NAME}")
print(f"  Gateway Name: {GATEWAY_NAME}")
print(f"  Region: {LAMBDA_REGION}")


### Step 1.4: Create IAM Role for Lambda Interceptor

Grant Lambda permissions to execute and write CloudWatch logs.

In [ ]:
# Create IAM role for Lambda interceptor using utils
print("Creating IAM role for Lambda interceptor...")

LAMBDA_ROLE_ARN = utils.create_lambda_role(
    role_name=LAMBDA_ROLE_NAME,
    description='Role for AgentCore Lambda Interceptor for schema translation'
)

print(f"  ARN: {LAMBDA_ROLE_ARN}")

### Step 1.5: Deploy Lambda Interceptor Function

Lambda intercepts tool requests and translates schema parameters before forwarding to targets.

In [ ]:
# Deploy Lambda interceptor using utils
print("Deploying Lambda interceptor...")

REQUEST_LAMBDA_ARN = utils.deploy_lambda_function(
    function_name=REQUEST_LAMBDA_FUNCTION_NAME,
    role_arn=LAMBDA_ROLE_ARN,
    lambda_code_path='src/lambda/st_request_interceptor.py',
    description='AgentCore Request Lambda Interceptor for schema translation',
    timeout=30,
    memory_size=256,
    region=LAMBDA_REGION
)

print(f"  ARN: {REQUEST_LAMBDA_ARN}")

In [ ]:
# Grant Gateway permission to invoke the Lambda interceptor
print("\nGranting Gateway permission to invoke Lambda...")

utils.grant_gateway_invoke_permission(
    function_name=REQUEST_LAMBDA_FUNCTION_NAME,
    region=LAMBDA_REGION
)

### Step 1.6: Create Amazon Cognito User Pool & App Client

Create Cognito user pool for Gateway authentication using OAuth client credentials flow.

In [ ]:
# Create Cognito User Pool and Client for Gateway authentication using utils
print("Creating Cognito User Pool and Client...")

cognito_client = boto3.client('cognito-idp', region_name=LAMBDA_REGION)

USER_POOL_NAME = f"gateway-pool-{DEPLOYMENT_ID}"
RESOURCE_SERVER_ID = 'gateway'
RESOURCE_SERVER_NAME = 'Gateway Resource Server'
SCOPES = [{'ScopeName': 'tools', 'ScopeDescription': 'Access to gateway tools'}]

# Create or get user pool
USER_POOL_ID = utils.get_or_create_user_pool(cognito_client, USER_POOL_NAME)
print(f"  Pool ID: {USER_POOL_ID}")

# Create or get resource server
utils.get_or_create_resource_server(cognito_client, USER_POOL_ID, RESOURCE_SERVER_ID, RESOURCE_SERVER_NAME, SCOPES)

# Wait for resource server to propagate
print("  Waiting for resource server to propagate...")
time.sleep(3)

# Create M2M client with client credentials flow
CLIENT_NAME = f"gateway-client-{DEPLOYMENT_ID}"
CLIENT_ID, CLIENT_SECRET = utils.get_or_create_m2m_client(
    cognito_client,
    USER_POOL_ID,
    CLIENT_NAME,
    RESOURCE_SERVER_ID,
    SCOPES=[f"{RESOURCE_SERVER_ID}/tools"]
)

print(f"✓ User Pool Client created: {CLIENT_NAME}")
print(f"  Client ID: {CLIENT_ID}")
print(f"  Client Secret: {CLIENT_SECRET[:20]}...")

# Construct OAuth URLs
POOL_DOMAIN = USER_POOL_ID.replace('_', '').lower()
COGNITO_DOMAIN = f"https://{POOL_DOMAIN}.auth.{LAMBDA_REGION}.amazoncognito.com"
DISCOVERY_URL = f"https://cognito-idp.{LAMBDA_REGION}.amazonaws.com/{USER_POOL_ID}/.well-known/openid-configuration"
TOKEN_URL = f"{COGNITO_DOMAIN}/oauth2/token"

print(f"\n✓ OAuth Configuration:")
print(f"  Discovery URL: {DISCOVERY_URL}")
print(f"  Token URL: {TOKEN_URL}")
print(f"  Scope: {RESOURCE_SERVER_ID}/tools")

### Step 1.7: Create Gateway with Request Interceptor

**Why REQUEST Interceptor?**  
The interceptor processes tool requests before execution, allowing us to translate schema parameters.

In [ ]:
# Create Gateway with Lambda interceptors
print("Creating Gateway with Lambda REQUEST interceptor...")

# Create IAM role for Gateway using utils
GATEWAY_ROLE_NAME = f"gateway-role-{DEPLOYMENT_ID}"
GATEWAY_ROLE_ARN = utils.create_agentcore_gateway_role_with_region(
    role_name=GATEWAY_ROLE_NAME,
    region=LAMBDA_REGION
)

# Create Gateway using boto3 client with interceptor configuration
gateway_client = boto3.client('bedrock-agentcore', region_name=LAMBDA_REGION)

print(f"\n  Creating Gateway with REQUEST interceptor:")
print(f"    Name: {GATEWAY_NAME}")
print(f"    Protocol: MCP")
print(f"    Auth: CUSTOM_JWT (Cognito)")
print(f"    Request Interceptor: {REQUEST_LAMBDA_ARN}")


gateway_control_plane_url = f"https://bedrock-agentcore-control.{LAMBDA_REGION}.amazonaws.com/gateways"

try:
    print(f"\n Creating gateway using {gateway_control_plane_url}")

    gateway_response = gateway_client.create_gateway(
        name=GATEWAY_NAME,
        protocolType="MCP",
        protocolConfiguration={
            "mcp": {
                "supportedVersions": ["2025-03-26"]
            }
        },

        interceptorConfigurations=[
            {
                "interceptor": {
                    "lambda": {
                        "arn": REQUEST_LAMBDA_ARN
                    }
                },
                "interceptionPoints": ["REQUEST"],  # Intercept requests for schema translation
                "inputConfiguration": {
                    "passRequestHeaders": True  # Pass Agent-ID header to interceptor
                }
            }
        ],

        authorizerType="CUSTOM_JWT",
        authorizerConfiguration={
            "customJWTAuthorizer": {
                "discoveryUrl": DISCOVERY_URL,
                "allowedClients": [CLIENT_ID]
            }
        },

        roleArn=GATEWAY_ROLE_ARN
    )
    
    status_code = gateway_response.get("ResponseMetadata", {}).get("HTTPStatusCode")
    if status_code not in [200, 202]:
        print(f"\n✗ Failed to create Gateway: {status_code}")
        print(f"  Response: {gateway_response.text}")
        raise Exception(f"Gateway creation failed: {gateway_response.text}")
    
    # gateway_data = gateway_response.json()
    GATEWAY_ID = gateway_response.get('gatewayId')
    
    print(f"\n✓ Gateway created successfully with REQUEST and RESPONSE interceptors")
    print(f"  ID: {GATEWAY_ID}")
    print(f"  Status: {gateway_response.get('status', 'CREATING')}")
    print(f"  Request Interceptor Lambda: {REQUEST_LAMBDA_ARN}")
    
    # Verify interceptor configuration in response
    if 'interceptorConfigurations' in gateway_response and gateway_response['interceptorConfigurations']:
        print(f"  ✓ Interceptor configuration confirmed in response!")
        print(f"    Interceptors: {len(gateway_response['interceptorConfigurations'])}")
        
        # Log details of each interceptor
        for i, config in enumerate(gateway_response['interceptorConfigurations']):
            points = config.get('interceptionPoints', [])
            print(f"    Interceptor {i+1}: {', '.join(points)}")
    else:
        print(f"  ⚠ Warning: No interceptor configuration in response")
    
except Exception as e:
    print(f"\n✗ Failed to create Gateway: {e}")
    raise


### Step 1.8: Wait for Gateway to be Ready

In [ ]:
# Wait for Gateway to be ready using signed requests
print("\nWaiting for Gateway to be ready...")

# get_gateway_url = f"{gateway_control_plane_url}/{GATEWAY_ID}"

max_attempts = 30
for attempt in range(max_attempts):
    try:
        response = gateway_client.get_gateway(gatewayIdentifier=GATEWAY_ID)
        status_code = response.get("ResponseMetadata", {}).get("HTTPStatusCode")

        if status_code == 200:
            # gateway_info = response.json()
            status = response.get('status', 'UNKNOWN')
            
            print(f"  [{attempt + 1}/{max_attempts}] Status: {status}")
            
            if status == 'READY':
                GATEWAY_URL = response.get('gatewayUrl')
                print(f"\n✓ Gateway is ready!")
                print(f"  URL: {GATEWAY_URL}")
                
                # Show interceptor configuration
                if 'interceptorConfigurations' in response:
                    interceptor_configs = response['interceptorConfigurations']
                    print(f"\n  Interceptor Configuration:")
                    for idx, config in enumerate(interceptor_configs):
                        print(f"    [{idx}] Interception Points: {config.get('interceptionPoints', [])}")
                        print(f"    [{idx}] Lambda ARN: {config.get('interceptor', {}).get('lambda', {}).get('arn', 'N/A')}")
                        print(f"    [{idx}] Pass Headers: {config.get('inputConfiguration', {}).get('passRequestHeaders', False)}")
                break
            elif status == 'FAILED':
                print(f"\n✗ Gateway creation failed")
                print(f"  Details: {response}")
                raise Exception("Gateway failed")
        else:
            print(f"  [{attempt + 1}/{max_attempts}] HTTP Error: {response.status_code}")
    except Exception as e:
        print(f"  [{attempt + 1}/{max_attempts}] Error: {e}")
    
    time.sleep(10)
else:
    print(f"\n⚠ Timeout waiting for Gateway")
    raise Exception("Gateway timeout")


### Step 1.9: Register Sample Inventory Tool with Gateway

Deploy an inventory tool Lambda and register it as a Gateway target.

In [ ]:
# Deploy inventory tool Lambda and register as Gateway target
print("="*80)
print("Deploying Inventory Tool Lambda and Registering with Gateway")
print("="*80)

import sys
import importlib

# Import tool module
sys.path.insert(0, str(Path.cwd()))
from src.tools import inventory_tool
inventory_tool = importlib.reload(inventory_tool)

print("✓ Tool module loaded")

# Create IAM role for tool Lambda using utils
print("\nCreating IAM role for tool Lambda...")
TOOL_ROLE_NAME = f"tool-lambda-role-{DEPLOYMENT_ID}"
TOOL_ROLE_ARN = utils.create_lambda_role(
    role_name=TOOL_ROLE_NAME,
    description='Role for inventory tool Lambda function'
)

# Deploy inventory tool Lambda using utils
print("\nDeploying inventory_tool...")

INVENTORY_FUNCTION_NAME = f"inventory-tool-{DEPLOYMENT_ID}"
INVENTORY_LAMBDA_ARN = utils.deploy_lambda_function(
    function_name=INVENTORY_FUNCTION_NAME,
    role_arn=TOOL_ROLE_ARN,
    lambda_code_path='src/tools/inventory_tool.py',
    description='Inventory tool for schema translation demo',
    timeout=30,
    memory_size=256,
    region=LAMBDA_REGION,
    environment_vars={'TOOL_NAME': 'inventory_tool'}
)

print(f"    ✓ Created: {INVENTORY_FUNCTION_NAME}")
print(f"    ARN: {INVENTORY_LAMBDA_ARN}")

# Get tool definition
tool_definition = getattr(inventory_tool, 'TOOL_DEFINITION', {
    "name": "inventory_tool",
    "description": "inventory_tool function"
})

deployed_tools = [{
    'tool_name': 'inventory_tool',
    'function_name': INVENTORY_FUNCTION_NAME,
    'lambda_arn': INVENTORY_LAMBDA_ARN,
    'tool_definition': tool_definition
}]

print(f"\n✓ Deployed {len(deployed_tools)} tool Lambda")

# Step 2: Register inventory tool as Gateway target
print(f"\n🎯 Step 2: Registering inventory tool as Gateway target...")
print("-" * 60)

print(f"\n  Registering inventory_tool...")

response = gateway_client.create_gateway_target(
    gatewayIdentifier=GATEWAY_ID,
    name="inventory-tool-target",
    targetConfiguration={
        "mcp": {
            "lambda": {
                "lambdaArn": INVENTORY_LAMBDA_ARN,
                "toolSchema": {
                    "inlinePayload": [tool_definition]
                }
            }
        }
    },
    credentialProviderConfigurations=[
        {"credentialProviderType": "GATEWAY_IAM_ROLE"}
    ]
)

INVENTORY_TARGET_ID = response['targetId']
print(f"    ✓ Target created: {INVENTORY_TARGET_ID}")

# Wait for target to be READY
print(f"    Waiting for target to be READY...")

for attempt in range(18):  # 3 minutes max
    response = gateway_client.get_gateway_target(
        gatewayIdentifier=GATEWAY_ID,
        targetId=INVENTORY_TARGET_ID
    )
    
    status = response.get('status', 'UNKNOWN')
    
    if status == 'READY':
        print(f"    ✓ Target is READY")
        break
    elif status == 'FAILED':
        print(f"    ✗ Target FAILED")
        print(f"    Details: {json.dumps(response, indent=6, default=str)}")
        raise Exception("Target creation failed")
    
    time.sleep(10)
else:
    raise Exception("Timeout waiting for target to be READY")

print(f"\n✅ Inventory tool registered and ready!")

# Store for cleanup
DEPLOYED_TOOL_FUNCTIONS = [INVENTORY_FUNCTION_NAME]
CREATED_TARGET_IDS = [INVENTORY_TARGET_ID]


---

## Part 2: Testing

### Step 2.1: List All Tools on the Gateway

In [ ]:
import requests
import json


def fetch_access_token(client_id, client_secret, token_url):
  response = requests.post(
    token_url,
    data="grant_type=client_credentials&client_id={client_id}&client_secret={client_secret}".format(client_id=client_id, client_secret=client_secret),
    headers={'Content-Type': 'application/x-www-form-urlencoded'}
  )

  return response.json()['access_token']

def list_tools(gateway_url, access_token):
  headers = {
      "Content-Type": "application/json",
      "Authorization": f"Bearer {access_token}"
  }

  payload = {
      "jsonrpc": "2.0",
      "id": "list-tools-request",
      "method": "tools/list"
  }

  response = requests.post(gateway_url, headers=headers, json=payload)
  return response.json()

access_token = fetch_access_token(CLIENT_ID, CLIENT_SECRET, TOKEN_URL)
tools = list_tools(GATEWAY_URL, access_token)
print(json.dumps(tools, indent=2))

### Step 2.2: Invoke Inventory Tool with Arguments

In [ ]:
# Helper function to invoke a specific tool through Gateway with REAL OAuth token
def invoke_tool_through_gateway(gateway_url, tool_name, arguments):
    """Make real MCP request to Gateway to invoke a specific tool with real OAuth token from Cognito."""
    import requests as req

    gateway_target_name = "inventory-tool-target"  # Set gateway target name here
    tool_name = "inventory_tool"

    # print(f"\n🌐 Invoking tool '{tool_name}' on agent: {agent_id}")
    print(f"   Gateway URL: {gateway_url}")

    # Get REAL OAuth token from Cognito using the credentials created earlier
    print(f"   Getting real OAuth token from Cognito...")
    print(f"   Using User Pool: {USER_POOL_ID}")
    print(f"   Using Client: {CLIENT_ID}")
    print(f"   Using Client: {CLIENT_SECRET}")

    try:
        # Get token using client credentials flow
        print(f"Requesting token from: {TOKEN_URL}")

        token_response = req.post(
            TOKEN_URL,
            headers={'Content-Type': 'application/x-www-form-urlencoded'},
            data={
                'grant_type': 'client_credentials',
                'client_id': CLIENT_ID,
                'client_secret': CLIENT_SECRET,
                'scope': 'gateway/tools'
            }
        )

        if token_response.status_code == 200:
            token_data = token_response.json()
            oauth_token = token_data['access_token']
            print(oauth_token)
            print(f"   ✓ Got real OAuth token from Cognito")
        else:
            error_msg = f"Failed to get token: {token_response.status_code} - {token_response.text}"
            print(f"   ✗ {error_msg}")
            return None, error_msg

    except Exception as e:
        error_msg = f"Token retrieval error: {e}"
        print(f"   ✗ {error_msg}")
        return None, error_msg

    # Default to empty arguments if none provided
    if arguments is None:
        arguments = {}

    # Construct MCP request for tools/call with tool name and inputs
    mcp_request = {
        "jsonrpc": "2.0",
        "method": "tools/call",
        "id": 1,
        "params": {
            "name": f"{gateway_target_name}___{tool_name}",
            "arguments": arguments
        }
    }

    headers = {
        "Authorization": f"Bearer {oauth_token}",
        "Content-Type": "application/json"
    }

    try:
        gateway_url = GATEWAY_URL
        response = req.post(gateway_url, headers=headers, json=mcp_request, timeout=30)

        if response.status_code == 200:
            data = response.json()
            print(f"   ✓ Tool '{tool_name}' invoked successfully")
            return data, None
        else:
            error = f"Status {response.status_code}: {response.text}"
            print(f"   ✗ {error}")
            return None, error
    except Exception as e:
        error = str(e)
        print(f"   ✗ Error: {error}")
        return None, error


## Invoke tool with an argument

- When tool is invoked using "item_id" argument, the lambda request interceptor changes the schema from "item_id" to "itemId"
- Also the request interceptor adds in an additional argument "location" as "East Warehouse" by default if location is not provided

In [ ]:
# Call the function to invoke the tool through the gateway
arguments = {"item_id": "ITEM-12345"}
result, error = invoke_tool_through_gateway(GATEWAY_URL, tool_name, arguments)

if error:
    print(f"Error invoking tool: {error}")
else:
    print("Tool invocation response:")
    print(result)

## Invoke tool with multiple arguments

- When tool is invoked using "item_id" argument, the lambda request interceptor changes the schema from "item_id" to "itemId"
- since location value is provided by the user, this time it will be passed to the tool and the default value wont be used

In [ ]:
# Call the function to invoke the tool through the gateway
arguments = {
    "item_id": "ITEM-12345",
    "location": "West Warehouse"
    }
result, error = invoke_tool_through_gateway(GATEWAY_URL, tool_name, arguments)

if error:
    print(f"Error invoking tool: {error}")
else:
    print("Tool invocation response:")
    print(result)

---

## Part 3: Cleanup

⚠️ **WARNING: This will DELETE all resources created in Part 1!**

Only run this section if you want to clean up everything.

In [ ]:
# Cleanup - Delete all created resources using utils
print("Starting cleanup...")

# 1. Delete gateway targets
if 'CREATED_TARGET_IDS' in globals() and 'GATEWAY_ID' in globals():
    utils.delete_gateway_targets(gateway_client, GATEWAY_ID, CREATED_TARGET_IDS)
    # Wait for target deletions to complete before deleting gateway
    time.sleep(5)

# 2. Delete gateway
if 'GATEWAY_ID' in globals():
    utils.delete_gateway(gateway_client, GATEWAY_ID)
    print("✓ Deleted gateway")

# 3. Delete Lambda functions (tools + interceptor)
lambda_functions_to_delete = []
if 'DEPLOYED_TOOL_FUNCTIONS' in globals():
    lambda_functions_to_delete.extend(DEPLOYED_TOOL_FUNCTIONS)
if 'REQUEST_LAMBDA_FUNCTION_NAME' in globals():
    lambda_functions_to_delete.append(REQUEST_LAMBDA_FUNCTION_NAME)

if lambda_functions_to_delete:
    utils.delete_lambda_functions(lambda_functions_to_delete, LAMBDA_REGION)

# 4. Delete IAM roles
if 'LAMBDA_ROLE_NAME' in globals():
    utils.delete_iam_role(LAMBDA_ROLE_NAME)
if 'DEPLOYMENT_ID' in globals():
    utils.delete_iam_role(f"tool-lambda-role-{DEPLOYMENT_ID}")
    utils.delete_iam_role(f"agentcore-{GATEWAY_NAME}-role")

# 5. Delete Cognito user pool
if 'USER_POOL_ID' in globals():
    utils.delete_cognito_user_pool(USER_POOL_ID, LAMBDA_REGION)

print("\n✓ Cleanup complete!")



---

# Summary

This notebook completed the full lifecycle:

1. ✅ **Setup** - Lambda, IAM Role, and Gateway
2. ✅ **Test** - Verified through AgentCore Gateway
3. ✅ **Cleanup** - Deleted all resources

## What We Demonstrated

- **Lambda REQUEST interceptor** that modifies Gateway requests
- **Transforming Schema and adding additional parameters** through the request chain
- **Complete resource lifecycle** management

## Next Steps

- Adding more schema translation use-cases
- Integrate with real AgentCore Runtime agents
- Monitor CloudWatch logs for debugging